In [12]:
import cv2
import torch

import torch
import torch.nn as nn
import numpy as np
import torchvision
from PIL import Image
from matplotlib import cm
from torch.utils.serialization import load_lua

ImportError: No module named serialization

In [2]:
## fer map
emotion_dict = {0: "Anger", 1: "Disgust", 2: "Fear", 3: "Happiness", 4: "Sadness", 5: "Surprise", 6: "Neutral"}


class Config:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

config = Config(
    num_classes = 7,
    width = 224,
    height = 224,
    num_epochs = 30,
    batch_size = 32,
    feat_dim = 11,
    lr_cent = 0.5,
    closs_weight = 0.5,
    ckp = True
)

In [8]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 7)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = F.dropout(out, p=0.5, training=self.training)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)
    
model = VGG('VGG19')
model.load_state_dict(torch.load('models/vgg.t7', map_location=torch.device('cpu')))
model.eval()


RuntimeError: Error(s) in loading state_dict for VGG:
	Missing key(s) in state_dict: "features.0.bias", "features.0.weight", "features.1.running_var", "features.1.bias", "features.1.weight", "features.1.running_mean", "features.3.bias", "features.3.weight", "features.4.running_var", "features.4.bias", "features.4.weight", "features.4.running_mean", "features.7.bias", "features.7.weight", "features.8.running_var", "features.8.bias", "features.8.weight", "features.8.running_mean", "features.10.bias", "features.10.weight", "features.11.running_var", "features.11.bias", "features.11.weight", "features.11.running_mean", "features.14.bias", "features.14.weight", "features.15.running_var", "features.15.bias", "features.15.weight", "features.15.running_mean", "features.17.bias", "features.17.weight", "features.18.running_var", "features.18.bias", "features.18.weight", "features.18.running_mean", "features.20.bias", "features.20.weight", "features.21.running_var", "features.21.bias", "features.

In [4]:
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Read the input image
frame = cv2.imread('test6_sad.png')

# Convert into grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
print("frame shape: ", frame.shape)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)


# Draw the rectangle around each face
for (x, y, w, h) in faces:
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
    roi_gray = gray[y:y + h, x:x + w]
    
    
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (224, 224)), 0), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    
    print("img shape: ", cropped_img.shape)
    print(type(cropped_img))
    
    cropped_img = torch.from_numpy(cropped_img)
    cropped_img = cropped_img.float()
    prediction = model(cropped_img)
        
    print(prediction)
    print(torch.argmax(prediction).item())
    cv2.putText(frame, emotion_dict[int(torch.argmax(prediction).item())], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)


    
# Display
cv2.imshow('frame', frame)
cv2.waitKey(0)
cv2.destroyWindow('frame')

('frame shape: ', (490, 640, 3))
('img shape: ', (1, 1, 224, 224))
<type 'numpy.ndarray'>
tensor([[  40.6555, -342.3951, -361.2148, -122.0296,  267.2000,   -0.6571,
          -64.9311]], grad_fn=<AddmmBackward>)
4


In [5]:
# # # To capture video from webcam. 
# cap = cv2.VideoCapture(0)
# # # To use a video file as input 
# # cap = cv2.VideoCapture('filename.mp4')

# # # Load the cascade
# face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# while True:
#     # Read the frame
#     ret, frame = cap.read()
    
#     # Convert to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#     # Detect the faces
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
#     # Draw the rectangle around each face
#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
#         roi_gray = gray[y:y + h, x:x + w]
# #         cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
#         cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (224, 224)), 0), 0)

#         cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)

#         cropped_img = torch.from_numpy(cropped_img)
#         cropped_img = cropped_img.float()
#         prediction = model(cropped_img)
        
#         print(prediction)
#         cv2.putText(frame, emotion_dict[int(torch.argmax(prediction).item())], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

#     # Display
#     cv2.imshow('frame', frame)
    
#     # Stop if q key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# # Release the VideoCapture object
# cap.release()
# cv2.destroyAllWindows()